# Note méthodologique : preuve de concept

## Dataset retenu

Le dataset utilisé dans cette preuve de concept est le jeu de données **IMDb**, très populaire dans le domaine de la classification de sentiments. Il contient **50 000 critiques de films**, chacune annotée par un label binaire :
- `0` pour une critique négative
- `1` pour une critique positive

Le dataset est équilibré, avec 25 000 critiques dans l’ensemble d’entraînement et 25 000 dans l’ensemble de test. Ce jeu est prétraité, ce qui signifie qu’il ne contient pas de balises HTML ni de symboles inutiles, et les textes sont en anglais.

Ce dataset est particulièrement adapté pour tester des algorithmes de NLP récents comme BERT ou pour évaluer des méthodes plus classiques comme le TF-IDF associé à une régression logistique.

L'objectif est d'entraîner un modèle capable de prédire la polarité d'une critique de film de manière fiable. Sa taille raisonnable et sa qualité font de ce dataset une référence pour des benchmarks de modèles de traitement du langage naturel.


## Les concepts de l’algorithme récent : Modern BERT

### Qu'est-ce que ModernBERT ?

ModernBERT fait référence à une famille de modèles BERT optimisés et compacts développés par Google pour des usages en production, comme par exemple : `google/bert_uncased_L-4_H-256_A-4`.

Ce modèle repose sur l'architecture **Transformer**, introduite dans le papier *Attention Is All You Need*. Il permet un traitement contextuel bidirectionnel, ce qui signifie qu’il prend en compte à la fois le contexte gauche et droit d’un mot dans une phrase.

### Architecture de ModernBERT

ModernBERT est une version allégée de BERT :
- **L = 4** : 4 couches de Transformer
- **H = 256** : taille de vecteur caché (embedding)
- **A = 4** : 4 têtes d’attention

Cette architecture permet de **réduire la taille** du modèle, tout en **préservant de bonnes performances** sur des tâches de classification de texte, avec un temps de calcul réduit par rapport à BERT original (12 couches, 768 dimensions).

### Intérêts du modèle

- Compatible avec des machines locales sans GPU puissant
- Moins coûteux en mémoire que BERT standard
- Plus performant qu’un simple modèle de type TF-IDF + régression
- Idéal pour une preuve de concept ou des applications embarquées


## La modélisation

Deux approches ont été comparées :

### 1. Modèle classique : TF-IDF + Régression Logistique
- **TF-IDF** extrait les mots importants du texte en réduisant l'impact des mots trop fréquents.
- Une **régression logistique** est ensuite entraînée pour classifier les critiques comme positives ou négatives.

### 2. Modèle récent : ModernBERT fine-tuné
- Utilisation du tokenizer de BERT pour découper les textes en sous-tokens (`WordPiece`).
- Conversion en tenseurs et **fine-tuning du modèle `google/bert_uncased_L-4_H-256_A-4`** sur le dataset IMDb.
- Optimisation avec `AdamW`, 2 epochs, batch size de 16, learning rate de 2e-5.
- Évaluation via `Trainer` de Hugging Face.

### Métrique utilisée
La **précision (accuracy)** est utilisée comme métrique principale, car le dataset est équilibré. Elle permet de mesurer la proportion correcte de prédictions.

### Améliorations proposées
- Augmenter la taille de l’échantillon (plus de 2 000 textes) pour une meilleure généralisation.
- Ajouter du dropout ou un scheduler de learning rate.
- Appliquer du **data augmentation** (synonymes, paraphrases) pour enrichir l'entraînement.
- Entraîner sur plus d’epochs (ex : 3 à 5).


## Synthèse des résultats

### Résultats obtenus

| Modèle                     | Accuracy |
|---------------------------|----------|
| TF-IDF + Régression Log.  | 89%      |
| ModernBERT (fine-tuné)    | **90%**  |

### Analyse

Le modèle ModernBERT surpasse légèrement le modèle TF-IDF en précision. Ce résultat est attendu : ModernBERT capture la **structure grammaticale et le contexte** des mots, alors que TF-IDF reste une approche purement statistique et indépendante du contexte.

ModernBERT :
- est plus performant sur des critiques longues ou ambigües.
- généralise mieux que les approches classiques.

TF-IDF :
- rapide et léger, mais limité dans sa compréhension sémantique.
- sensible au vocabulaire utilisé, pas au sens global.

### Conclusion

ModernBERT constitue une amélioration nette sur la qualité de prédiction, sans nécessiter une architecture lourde. Même sur un PC sans GPU, ses performances sont compétitives avec un temps d'entraînement raisonnable (~10 minutes).


## Analyse de la feature importance globale et locale

L’interprétabilité est une problématique importante dans les modèles NLP modernes.

### TF-IDF + Régression logistique
Ce modèle est facilement interprétable :
- Chaque poids du vecteur de coefficients correspond à un mot.
- On peut extraire les **mots les plus influents** pour la classe positive ou négative (ex: *"boring"*, *"great"*).

### ModernBERT
En revanche, ModernBERT est un modèle **boîte noire**. L’interprétation directe des poids est difficile.

> **Remarque :** Nous n'avons pas encore intégré de méthode d’explicabilité comme SHAP ou LIME dans le notebook.

### Pistes d’amélioration
- Utiliser la bibliothèque **SHAP** pour obtenir l’importance locale des mots (par exemple sous forme de heatmap sur les tokens).
- Ajouter un visualiseur des attentions (`bertviz`) pour montrer quelles parties du texte influencent le plus la sortie du modèle.


## Limites et améliorations possibles

### Limites du modèle ModernBERT

- Précision encore améliorable : 90% reste perfectible pour certaines applications sensibles.
- Longs textes tronqués à 512 tokens : perte de contexte potentiel.
- Modèle difficilement interprétable sans outils externes.
- Nécessite du fine-tuning même pour un bon résultat.

### Améliorations envisageables

- **Data augmentation** : enrichir le jeu de données avec des paraphrases ou synonymes.
- **Augmenter le nombre d’epochs** à 4 ou 5.
- **Hyperparameter tuning** : recherche plus fine des meilleurs paramètres.
- **Utiliser un modèle plus récent** (ex: `deberta`, `roberta-base`) pour pousser les performances plus loin.
- **Ajout de SHAP ou LIME** pour une interprétation locale des prédictions.

Malgré ses limitations, ModernBERT est un excellent compromis entre performance et coût de calcul pour les projets en environnement contraint.
